In [41]:
import os
import glob
import h5py

def copy_items(name, obj, destination_group):
    if isinstance(obj, h5py.Group):
        if name not in destination_group:
            new_group = destination_group.create_group(name)
            obj.visititems(lambda subname, subobj: copy_items(subname, subobj, new_group))
    elif isinstance(obj, h5py.Dataset):

        if name in destination_group:
            existing_dataset = destination_group[name]
            existing_shape = existing_dataset.shape
            existing_dataset.resize((existing_shape[0] + obj.shape[0],) + existing_shape[1:])
            existing_dataset[-obj.shape[0]:] = obj[:]
        else:
            chunks = (1,) + obj.shape[1:]
            destination_group.create_dataset(name, shape=obj.shape, dtype=obj.dtype, chunks=chunks, maxshape=(None,) + obj.shape[1:], data=obj[:])

def concatenate_h5_in_folder(output_file, folder='ML_ana_h5'):
    files_to_concatenate = glob.glob(os.path.join(folder, '*.h5'))


    with h5py.File(output_file, 'w') as output_h5:
        for file_path in files_to_concatenate:
            with h5py.File(file_path, 'r') as input_h5:
                input_h5.visititems(lambda name, obj: copy_items(name, obj, output_h5))

output_file_path = 'combined_out.h5'
concatenate_h5_in_folder(output_file_path)
